# X' (X Prime) Experiment 
- Adding the location channels in the beginning of the image, and not at each layer step. 

In [1]:
# Torch
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 
from torchsummary import summary


# Train + Data 
import sys 
sys.path.append('../Layers')
from Conv1d_NN import *
from Conv2d_NN import *
from Conv1d_NN_spatial import * 
from Conv2d_NN_spatial import * 
from ConvNN_CNN_Branching import *

sys.path.append('../Data')
from CIFAR10 import * 


sys.path.append('../Models')
from models_2d import *

sys.path.append('../Train')
from train2d import * 


/Users/mingikang/miniforge3/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cifar10 = CIFAR10()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
## CNN
class CNN_Location_Before(nn.Module):
    def __init__(self, in_ch=3, num_classes=10, kernel_size=3):
        super(CNN_Location_Before, self).__init__()
        
        
        self.conv1 = nn.Conv2d(in_ch+2, 16, kernel_size=kernel_size, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=kernel_size, stride=1, padding=1)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(32768, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

        self.relu = nn.ReLU()
        self.to("mps")
        self.name = "CNN"

    def forward(self, x):
        x_coordinates = self.coordinate_channels(x.shape, x.device)
        x = torch.cat((x, x_coordinates), dim=1)
        
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.flatten(x)

        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x
    
    def summary(self, input_size = (3, 32, 32)): 
        self.to("cpu")
        print(summary(self, input_size))
        self.to("mps")
            
    def coordinate_channels(self, tensor_shape, device):
        x_ind = torch.arange(0, tensor_shape[2])
        y_ind = torch.arange(0, tensor_shape[3])
        
        x_grid, y_grid = torch.meshgrid(x_ind, y_ind, indexing='ij')
        
        x_grid = x_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
        y_grid = y_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
        
        xy_grid = torch.cat((x_grid, y_grid), dim=1)
        xy_grid_normalized = F.normalize(xy_grid, p=2, dim=1)
        return xy_grid_normalized.to(device)


In [4]:
class ConvNN_2D_K_All_Location_Before(nn.Module):
    def __init__(self, in_ch=3, num_classes=10, K=9):
        super(ConvNN_2D_K_All_Location_Before, self).__init__()
        
        self.conv1 = Conv2d_NN(in_ch+2, 16, K=K, stride=K, shuffle_pattern="BA", shuffle_scale=2, samples="all")
        self.conv2 = Conv2d_NN(16, 32, K=K, stride=K, shuffle_pattern="BA", shuffle_scale=2, samples="all")

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(32768, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

        self.relu = nn.ReLU()
        self.to("mps")
        self.name = "ConvNN_2D_K_All"

    def forward(self, x):
        x_coordinates = self.coordinate_channels(x.shape, x.device)
        x = torch.cat((x, x_coordinates), dim=1)
        
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.flatten(x)

        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x
    
    def summary(self, input_size = (3, 32, 32)): 
        self.to("cpu")
        print(summary(self, input_size))
        self.to("mps")
        
    def coordinate_channels(self, tensor_shape, device):
        x_ind = torch.arange(0, tensor_shape[2])
        y_ind = torch.arange(0, tensor_shape[3])
        
        x_grid, y_grid = torch.meshgrid(x_ind, y_ind, indexing='ij')
        
        x_grid = x_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
        y_grid = y_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
        
        xy_grid = torch.cat((x_grid, y_grid), dim=1)
        xy_grid_normalized = F.normalize(xy_grid, p=2, dim=1)
        return xy_grid_normalized.to(device)

In [5]:
class Branching_ConvNN_2D_Spatial_K_N_Location_Before(nn.Module):
    def __init__(self, in_ch=3, channel_ratio=(16, 16), num_classes=10, kernel_size=3, K=9, N = 8, location_channels = False):
        
        super(Branching_ConvNN_2D_Spatial_K_N_Location_Before, self).__init__()
        self.conv1 = ConvNN_CNN_Spatial_BranchingLayer(in_ch+2, 16, 
            channel_ratio=channel_ratio,kernel_size=kernel_size, K=K, samples=N, location_channels=location_channels)
        self.conv2 = ConvNN_CNN_Spatial_BranchingLayer(16, 32, channel_ratio=(channel_ratio[0] *2, channel_ratio[1]*2),kernel_size=kernel_size, K=K, samples=N, location_channels=location_channels)
        
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(32768, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

        self.relu = nn.ReLU()
        self.to("mps")
        self.name = "Branching_ConvNN_2D_Spatial_K_N"

    def forward(self, x):
        x_coordinates = self.coordinate_channels(x.shape, x.device)
        x = torch.cat((x, x_coordinates), dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = self.flatten(x)

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def summary(self, input_size = (3, 32, 32)): 
        self.to("cpu")
        print(summary(self, input_size))
        self.to("mps")
        
    def coordinate_channels(self, tensor_shape, device):
        x_ind = torch.arange(0, tensor_shape[2])
        y_ind = torch.arange(0, tensor_shape[3])
        
        x_grid, y_grid = torch.meshgrid(x_ind, y_ind, indexing='ij')
        
        x_grid = x_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
        y_grid = y_grid.float().unsqueeze(0).expand(tensor_shape[0], -1, -1).unsqueeze(1)
        
        xy_grid = torch.cat((x_grid, y_grid), dim=1)
        xy_grid_normalized = F.normalize(xy_grid, p=2, dim=1)
        return xy_grid_normalized.to(device)


In [6]:
# CNN with Location Channels added before 
cnn_location_before = CNN_Location_Before()

cnn_location_before.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_location_before.parameters(), lr=0.001)
num_epochs = 10 
train_model(cnn_location_before, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(cnn_location_before, cifar10.test_loader)

Epoch 1, Time: 30.6076557636261, Loss: 1.320821599277389
Epoch 2, Time: 34.71222472190857, Loss: 0.804229320124592
Epoch 3, Time: 34.860244035720825, Loss: 0.42631748023316685
Epoch 4, Time: 34.507848024368286, Loss: 0.14106527700915436
Epoch 5, Time: 34.50970196723938, Loss: 0.06549997788513331
Epoch 6, Time: 34.5172758102417, Loss: 0.049463629004452614
Epoch 7, Time: 34.6102077960968, Loss: 0.05258563129365911
Epoch 8, Time: 36.005460023880005, Loss: 0.0432165561500988
Epoch 9, Time: 42.281323194503784, Loss: 0.04389226100365262
Epoch 10, Time: 51.81594204902649, Loss: 0.03636969356222109

 Average epoch time: 36.842788338661194
Accuracy on test set: 64.05%


64.05

In [7]:
# ConNN 2D K = 9, All Samples with Location Channels added before
convNN_k_all_location_before = ConvNN_2D_K_All_Location_Before()

convNN_k_all_location_before.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(convNN_k_all_location_before.parameters(), lr=0.001)
num_epochs = 10 
train_model(convNN_k_all_location_before, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(convNN_k_all_location_before, cifar10.test_loader)

Epoch 1, Time: 88.54998207092285, Loss: 1.4418080857647655
Epoch 2, Time: 102.55355286598206, Loss: 1.0979220754350238
Epoch 3, Time: 105.41861128807068, Loss: 0.8576833204463925
Epoch 4, Time: 107.13352918624878, Loss: 0.5801344805056482
Epoch 5, Time: 104.16446471214294, Loss: 0.33415084219802066
Epoch 6, Time: 99.26240801811218, Loss: 0.20348460427330584
Epoch 7, Time: 101.254225730896, Loss: 0.15475174778467402
Epoch 8, Time: 97.87570118904114, Loss: 0.12964877465625516
Epoch 9, Time: 93.44150996208191, Loss: 0.1318924545585547
Epoch 10, Time: 92.4614040851593, Loss: 0.09777499881991045

 Average epoch time: 99.21153891086578
Accuracy on test set: 57.81%


57.81

In [8]:
# Branching ConvNN 2D Spatial Sampling with Location Channels added before
branching_convNN_spatial_k_n_location_before = Branching_ConvNN_2D_Spatial_K_N_Location_Before()

branching_convNN_spatial_k_n_location_before.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(branching_convNN_spatial_k_n_location_before.parameters(), lr=0.001)
num_epochs = 10 
train_model(branching_convNN_spatial_k_n_location_before, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(branching_convNN_spatial_k_n_location_before, cifar10.test_loader)

Epoch 1, Time: 94.06807804107666, Loss: 1.4150934026521795
Epoch 2, Time: 103.46013283729553, Loss: 0.9707661204020995
Epoch 3, Time: 103.36579298973083, Loss: 0.6381737245699329
Epoch 4, Time: 95.47790884971619, Loss: 0.27994981874018676
Epoch 5, Time: 96.22189211845398, Loss: 0.1296414005298577
Epoch 6, Time: 109.62095499038696, Loss: 0.11480781364330875
Epoch 7, Time: 93.30559086799622, Loss: 0.08681446669028972
Epoch 8, Time: 89.27326512336731, Loss: 0.07692564570683214
Epoch 9, Time: 100.74316811561584, Loss: 0.07736872752864614
Epoch 10, Time: 136.39051604270935, Loss: 0.06751669987137286

 Average epoch time: 102.19272999763488
Accuracy on test set: 65.72%


65.72